In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import *
from sklearn import cross_validation

/anaconda/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = read_csv("/Users/mrigank/Desktop/python_projects/ML_project/letter-recognition.data.csv", sep=',', header=None)

In [3]:
data = np.array(data, dtype=object)
data

array([['T', 2, 8, ..., 8, 0, 8],
       ['I', 5, 12, ..., 8, 4, 10],
       ['D', 4, 11, ..., 7, 3, 9],
       ..., 
       ['T', 6, 9, ..., 12, 2, 4],
       ['S', 2, 3, ..., 9, 5, 8],
       ['A', 4, 9, ..., 7, 2, 8]], dtype=object)

In [5]:
# seperate out X and y
X = data[:,1:]
y = data[:,0:1]
print("size of X is:", X.shape, "size of y is:", y.shape)
# insert the 1's to X
## adding biases to original X
X = np.insert(X,0,1,axis=1)
# X = np.concatenate((np.ones(shape = (X.shape[0],1)),X),axis=1)
print("size of X is:", X.shape, "size of y is:", y.shape)

size of X is: (20000, 16) size of y is: (20000, 1)
size of X is: (20000, 17) size of y is: (20000, 1)


In [88]:
# 25% of the data is kept apart for cross-validation and testing
## sklearn function is used to do so
X_train, X_t_cv, y_train, y_t_cv = cross_validation.train_test_split(X, y, test_size=0.25)
# data seperated for test & cross-validation
X_cv, X_test, y_cv, y_test = cross_validation.train_test_split(X_t_cv, y_t_cv, test_size=0.5)
X_train.shape
# train: 15000 : cv: 2500 : test: 2500 observations

(15000, 16)

In [30]:
## y labels are formatted into a array of 26 binary numbers, for 26 different classes
temp = np.zeros(shape = (y_train.shape[0],26))
temp
#y_train.append(temp)
u = np.unique(y_train)
for i in range(len(y_train)):
    for j in range(len(temp[0])):
        if (y_train[i] == u[j]):
            temp[i,j]=1
            break

In [89]:
temp.shape

(15000, 26)

In [90]:
y_train[1]

array(['Y'], dtype=object)

In [131]:
class Neural_Network(object):
    def __init__(self): 
        #Define Hyperparameters
        self.inputLayerSize = 16
        self.outputLayerSize = 26
        self.hiddenLayerSize = 32
        
        self.W1 = np.random.rand(self.inputLayerSize, self.hiddenLayerSize)
        self.W2 = np.random.rand(self.hiddenLayerSize, self.outputLayerSize)
        self.b2 = np.random.rand(1)
        self.b2 = np.random.rand(1)
        
    def sigmoid(self, X):
        return 1/(1 + np.exp(-np.float32(X)))
    
    def forward(self, X):
        self.z2 = np.dot(X, self.W1)# + self.b1
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.sigmoid(self.a2), self.W2)# + self.b2
        self.yHat = self.sigmoid(self.z3)
        
        return self.yHat
    
    def sigmoidPrime(self, z):
        #Derivative of sigmoid function
        return np.exp(-np.float32(z))/((1+np.exp(-np.float32(z)))**2)
    
    def costFunction(self, X, y):
        self.yHat = self.forward(X)
        
        J = 1/2 * ((y - self.yHat)**2)
        return J
    
    def costFunctionPrime(self, X, y):
        self.yHat = self.forward(X)
        
        delta3 = np.multiply((-(y-self.yHat)), self.sigmoidPrime(self.z3))
        dJdW2 = np.dot(self.a2.T, delta3)
        
        delta2 = np.dot(delta3, self.W2.T) * self.sigmoidPrime(self.z2)
        dJdW1 = np.dot(X.T, delta2)
        
        return dJdW1, dJdW2

In [116]:
NN = Neural_Network()
cost1 = NN.costFunction(X_train,y=temp)
cost1.shape

(15000, 26)

In [117]:
#yHat = forward(self, X_train)
dJdW1, dJdW2 = NN.costFunctionPrime(X_train, y = temp)

In [119]:
dJdW1.shape
NN.W1[0][0]

0.50855279182702562

In [120]:
#dJdW2

NN.W1[0][0] -= learningRate*dJdW1[0][0]
NN.W1[0][0]

0.50855279182655622

In [123]:
learningRate = 0.1

NN.W1 = NN.W1 + learningRate*dJdW1
NN.W2 = NN.W2 + learningRate*dJdW2
cost2 = NN.costFunction(X_train, y = temp)

In [108]:
NN.W1 = NN.W1 - learningRate * dJdW1
NN.W2 = NN.W2 - learningRate * dJdW2
cost3 = NN.costFunction(X_train, y = temp)

In [109]:
np.sum(cost1, axis=0)

array([ 7204.2955968 ,  7226.62265693,  7227.60881878,  7194.92109083,
        7200.23734235,  7208.51339872,  7200.99313259,  7236.77914603,
        7210.45702214,  7214.87270652,  7218.90361628,  7218.94664465,
        7210.48108972,  7201.93818487,  7206.92611404,  7201.96909241,
        7199.45142996,  7213.41572847,  7221.95178803,  7201.90070676,
        7197.14993682,  7218.92082762,  7220.22456251,  7190.54567347,
        7210.47077504,  7233.19992269])

In [110]:
np.sum(cost2, axis=0)

array([ 7204.30418511,  7226.65367104,  7227.64328364,  7194.92109083,
        7200.25279266,  7208.71793223,  7200.99313259,  7236.79122389,
        7210.45874125,  7214.87786704,  7218.90533742,  7218.94836578,
        7210.48108972,  7201.93818487,  7206.92783232,  7201.96909241,
        7201.00610813,  7213.41744829,  7221.95178803,  7201.97969521,
        7197.17739243,  7218.92254875,  7220.2417772 ,  7190.59196266,
        7210.47077504,  7233.22061747])

In [111]:
np.sum(cost3, axis=0)

array([ 7204.2955968 ,  7226.62265693,  7227.60881878,  7194.92109083,
        7200.23734235,  7208.51339872,  7200.99313259,  7236.77914603,
        7210.45702214,  7214.87270652,  7218.90361628,  7218.94664465,
        7210.48108972,  7201.93818487,  7206.92611404,  7201.96909241,
        7199.45142996,  7213.41572847,  7221.95178803,  7201.90070676,
        7197.14993682,  7218.92082762,  7220.22456251,  7190.54567347,
        7210.47077504,  7233.19992269])

In [124]:
learningRate*dJdW1[0][0]

4.693595279920857e-12